In [5]:
import spacy
import csv
import pandas as pd
import math
from spacy.tokens import DocBin
from spacy import displacy

In [ ]:
df = pd.read_csv('2complete.csv')
# specs_df = pd.read_csv('table_data_refrigerator.csv')
df = df.sample(frac = 1)
# train_df = df.sample(frac=0.6, random_state=200)
# test_df = df.drop(train_df.index)
# test_df

In [ ]:
data=[]
for ind in df.index:
    id = df['ID'][ind]
    title = df['Title'][ind]
    brand = df['Brand'][ind]
    model = df['Model'][ind]
    brand_start = title.find(brand)
    model_start = title.find(model)
    data.append({'ID':id, 'Title': title, 'Brand': brand, 'Model': model, 'brand_start': brand_start, 'brand_end': brand_start + len(brand), 'model_start': model_start, 'model_end': model_start + len(model)})
train = data[:math.floor(len(data)*0.6)]
test = data[math.floor(len(data)*0.6):]

In [ ]:
# train_df = df.sample(frac=0.6, random_state=200)
# test_df = df.drop(train_df.index)

In [ ]:
training_data = {'classes':['BRAND', 'MODEL'], 'annotations':[]}
for ex in train:
    temp_dict = {}
    temp_dict['text'] = ex['Title']
    temp_dict['entities'] = []
    temp_dict['entities'].append((ex['brand_start'], ex['brand_end'], 'BRAND'))
    temp_dict['entities'].append((ex['model_start'], ex['model_end'], 'MODEL'))
    training_data['annotations'].append(temp_dict)
training_data['annotations'][0]

In [ ]:
nlp = spacy.blank('en')
doc_bin = DocBin()

for training_example  in (training_data['annotations']): 
    text = training_example['text']
    labels = training_example['entities']
    doc = nlp.make_doc(text) 
    ents = []
    for start, end, label in labels:
        span = doc.char_span(start, end, label=label, alignment_mode="contract")
        if span is None:
            print("Skipping entity")
        else:
            ents.append(span)
    doc.ents = ents 
    doc_bin.add(doc)

doc_bin.to_disk("training_data.spacy") # save the docbin object

In [ ]:
nlp_ner = spacy.load("model-best")

# doc = nlp_ner("MarQ by Flipkart 468 L Frost Free Side by Side Refrigerator (468ASMQS)")
# for ent in doc.ents:
#     print(ent.text, ent.label_)
# colors = {"MODEL": "#F67DE3", "BRAND": "#7DF6D9"}
# options = {"colors": colors} 

# spacy.displacy.render(doc, style="ent", options= options, jupyter=True)
correct_brand = 0
incorrect_brand = 0
correct_model = 0
incorrect_model = 0
for test_ex in test:
    title = test_ex['Title']
    brand = test_ex['Brand']
    model = test_ex['Model']
    doc = nlp_ner(title)
    predicted_brand=''
    predicted_model=''
    for ent in doc.ents:
        if(ent.label_ == 'BRAND'):
            predicted_brand = ent.text
        if(ent.label_ == 'MODEL'):
            predicted_model = ent.text
    if(predicted_brand == brand):
        correct_brand += 1
    else:
        incorrect_brand += 1
    
    if(predicted_model == model):
        correct_model += 1
    else:
        incorrect_model += 1
print(correct_brand, incorrect_brand, correct_model, incorrect_model)

In [ ]:
doc = nlp_ner("abc 236 L 2 door Samsung")
for ent in doc.ents:
    print(ent.text, ent.label_)
colors = {"MODEL": "#F67DE3", "BRAND": "#7DF6D9"}
options = {"colors": colors} 

spacy.displacy.render(doc, style="ent", options= options, jupyter=True)